(fs:dl_track_free)=
# Disk Layout:Tracking Free Space

So, we know how to keep track of blocks used in a file, how do we represent blocks that are free?  

If you recall the approach used in [FAT file system](fs:dl_track_used:ll) to represent used blocks, the same data structure was also used to represent used blocks.  When allocating a block the file system needed to search through the linked list until it found an entry marked as free; an extreemly inefficient approach.


```{figure} ../images/fs/free_linkedlist.png
---
width: 80% 
name: fs:free:ll
---
Free list as a linked list of blocks
```
The original Unix file system used a free list to store a list of unused blocks (see {numref}`fs:fs:free:ll`);  blocks were allocated from the head of this list for new files, and returned to the head when freed. Implementing this results in a large initialization cost; if you have a 1TB drive, it means you will start off by having 4M 4KB blocks to hold the free list to start off with, however, it compresses to a small number of blocks for the free list when the disk becomes full.  

The bigger disadvantage of this approach is that, as files were created and deleted this list became randomized, so that blocks allocated for a file are rarely sequential and disk seeks were needed for nearly every block read from or written to disk. Originally this wasn't a significant problem, because early Unix systems ran on machines with fast disks and excruciatingly slow CPUs.  


```{figure} ../images/fs/free_bitvector.png
---
width: 80% 
name: fs:free:bv
---
Free blocks recorded as bits
```

As computers got faster and users started noticing that seek times where killing performance, file systems started using an *allocation bitmap* as shown in {numref}`fs:free:bv`. It
keeeps a boolean array with one bit for each disk block; if the block is
allocated the corresponding bit is set to '1', and cleared to '0' if it
is freed. To allocate a block you read a portion of this bitmap into
memory and scan for a '0' bit, changing it to '1' and writing it back.
When you extend a file you begin the search at the bit corresponding to
the last block in the file; in this way if a sequential block is
available it will be allocated. This method results in files being
allocated in a mostly sequential fashion, reducing disk seeks and
greatly improving performance. Note, when disk is fragmented, each block may have just a few bits available, so this approach becomes expensive to find a block.  However, since a single page of 4096 bytes can represent 132MB ($4096*8*4096), you can efficiently cache these blocks in memory. 